In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('zot.csv')
df.columns

Index(['Unnamed: 0', 'key', 'version', 'itemType', 'creators', 'tags', 'DOI',
       'url', 'volume', 'pages', 'title', 'publicationTitle', 'abstractNote',
       'date', 'ISSN', 'language', 'journalAbbreviation', 'libraryCatalog',
       'accessDate', 'bib', 'index', 'tURL', 'fail', 'perf', 'year', 'author',
       'rights', 'issue', 'shortTitle', 'numPages', 'extra', 'ISBN',
       'publisher', 'proceedingsTitle', 'conferenceName', 'series', 'place',
       'university', 'thesisType', 'bookTitle', 'edition', 'institution',
       'reportType', 'reportNumber', 'websiteType', 'websiteTitle',
       'seriesNumber', 'archiveLocation', 'callNumber', 'DOI_null'],
      dtype='object')

In [18]:
df['index'] = df['index'].fillna(method='ffill')+df.groupby(df['index'].notnull().cumsum()).cumcount()

In [3]:
adf = pd.read_json('google_organic_citations_short_name_ges_disc.json', 
                                orient='records')
adf.columns

Index(['result_id', 'link', 'pub_doi', 'index', 'author', 'year', 'title',
       'ShortNames'],
      dtype='object')

In [19]:
df['index']

0          0.0
1          1.0
2          2.0
3          3.0
4          4.0
         ...  
5362    5369.0
5363    5370.0
5364    5371.0
5365    5372.0
5366    5373.0
Name: index, Length: 5367, dtype: float64

In [20]:
fdf = df[['index', 
    'DOI']].merge(adf[['index', 'pub_doi']], 
                    on='index', how='inner')

fdf.rename(columns = {list(fdf)[1]: 'translator', list(fdf)[2]: 'original'}, inplace = True)
fdf['original'] = fdf['original'].replace('', np.nan)
fdf['translator_NA'] = (~fdf['translator'].isna()).astype(int)
fdf['original_NA'] = (~fdf['original'].isna()).astype(int)
fdf

,index,translator,original,translator_NA,original_NA
0,0.0,10.1088/1742-6596/869/1/012070,10.1088/1742-6596/869/1/012070,1,1
1,1.0,10.3390/rs12172821,NaN,1,0
2,2.0,NaN,NaN,0,0
3,3.0,10.1016/j.rse.2019.111269,NaN,1,0
4,4.0,10.1175/MWR-D-16-0234.1,10.1175/mwr-d-16-0234.1,1,1
...,...,...,...,...,...
5362,5369.0,NaN,NaN,0,0
5363,5370.0,10.1016/j.gloplacha.2019.102994,NaN,1,0
5364,5371.0,NaN,NaN,0,0
5365,5372.0,10.1016/j.epsl.2017.11.004,NaN,1,0


In [21]:
pd.crosstab(fdf.original_NA, fdf.translator_NA, normalize='all')

translator_NA,0,1
original_NA,,
0,0.212595,0.294019
1,0.095584,0.397801


In [22]:
pd.crosstab(fdf.original_NA, fdf.translator_NA)

translator_NA,0,1
original_NA,,
0,1141,1578
1,513,2135


when both are not NA, how often are they the same

In [23]:
cdf = fdf[fdf['translator_NA'] == 1][fdf['original_NA'] == 1]
if type(cdf['translator'].iloc[0]) == str:
    cdf['translator'] = cdf['translator'].apply(lambda x: x.lower())
    cdf['original'] = cdf['original'].apply(lambda x: x.lower())
(cdf['translator'] == cdf['original']).sum() / len(cdf)

/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_4299/3071723300.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cdf = fdf[fdf['translator_NA'] == 1][fdf['original_NA'] == 1]
/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_4299/3071723300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf['translator'] = cdf['translator'].apply(lambda x: x.lower())
/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_4299/3071723300.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

0.9850117096018736

In [24]:
cdf[cdf['translator'] != cdf['original']]

,index,translator,original,translator_NA,original_NA
334,335.0,10.2166/wcc.2022.420,10.2166/wcc.2022.420/87051,1,1
1065,1068.0,10.1175/1520-0477(2000)081<2035:eopsse>2.3.co;2,10.1175/1520-0477_2000_081_2035_eopsse_2_3_co_2,1,1
1075,1078.0,10.1175/1520-0426(2004)021<0624:teorio>2.0.co;2,10.1175/1520-0426_2004_021_0624_teorio_2_0_co_2,1,1
1104,1107.0,10.1175/1520-0450(2003)042<1519:ttprvo>2.0.co;2,10.1175/1520-0450_2003_042.1519_ttprvo_2.0.co_2,1,1
1106,1109.0,10.1175/1520-0450(2003)042<0769:cotpra>2.0.co;2,10.1175/1520-0450_2003_042_0769_cotpra_2.0.co_2,1,1
1120,1123.0,10.1175/1520-0450(2001)040<0331:dvosro>2.0.co;2,10.1175/1520-0450_2001_040_0331_dvosro_2.0.co_2,1,1
1306,1310.0,10.2166/ws.2021.387,10.2166/ws.2021.387/85261,1,1
1455,1459.0,10.2166/wcc.2022.410,10.2166/wcc.2022.410/86339,1,1
2494,2498.0,10.1175/1520-0442(2001)014<3566:rvitco>2.0.co;2,10.1175/1520-0442_2001_014_3566_rvitco_2.0.co_2,1,1
2546,2550.0,10.1175/1520-0450(2004)043<0562:roppfm>2.0.co;2,10.1175/1520-0450_2004_043_0562_roppfm_2.0.co_2,1,1


In [26]:
export_df = df.merge(adf.add_suffix('_o'), 
                    left_on='index', right_on='index_o', how='left')

export_df

,Unnamed: 0,key,version,itemType,creators,tags,DOI,url,volume,pages,...,callNumber,DOI_null,result_id_o,link_o,pub_doi_o,index_o,author_o,year_o,title_o,ShortNames_o
0,0,XZFWVNZT,0.0,journalArticle,"[{'firstName': 'G.', 'lastName': 'Abdelaziz', ...",[],10.1088/1742-6596/869/1/012070,https://doi.org/10.1088/1742-6596/869/1/012070,869.0,12070,...,NaN,DOI,bZ5KnGFtrEAJ,https://iopscience.iop.org/article/10.1088/174...,10.1088/1742-6596/869/1/012070,0,G Abdelaziz,(2017),Search for Best Astronomical Observatory Sites...,"[AIRX3STD, MAT1NXSLV, MAT1NXFLX]"
1,0,6EUEUZ3I,0.0,journalArticle,"[{'firstName': 'Zhong', 'lastName': 'Liu', 'cr...",[{'tag': 'tropical meteorology and climatology...,10.3390/rs12172821,https://www.mdpi.com/2072-4292/12/17/2821,12.0,2821,...,NaN,DOI,9n-nmxo8geAJ,https://www.mdpi.com/813170,,1,Z Liu,(2020),NASA global satellite and model data products ...,"[AIRX3STD, OMAEROe, NLDAS_NOAH0125_H, OMDOAO3e..."
2,0,ZRGZ7FQD,0.0,journalArticle,"[{'firstName': 'Abhay', 'lastName': 'Devasthal...",[],NaN,https://www.jstor.org/stable/26243653,97.0,ES255-ES263,...,NaN,Null,ixJUtElCtKAJ,https://www.jstor.org/stable/26243653,,2,A Devasthale,(2016),SUPPLEMENT: A DECADE OF SPACEBORNE OBSERVATION...,[AIRX3STD]
3,0,3HST9T9G,0.0,journalArticle,"[{'firstName': 'Guanheng', 'lastName': 'Zheng'...","[{'tag': 'Satellite remote sensing', 'type': 1...",10.1016/j.rse.2019.111269,https://www.sciencedirect.com/science/article/...,231.0,111269,...,NaN,DOI,tFBynLLhJPsJ,https://www.sciencedirect.com/science/article/...,,3,G Zheng,(2019),Satellite-based simulation of soil freezing/th...,[AIRX3STD]
4,0,E8NR9S8V,0.0,journalArticle,"[{'firstName': 'Linette N.', 'lastName': 'Bois...",[],10.1175/MWR-D-16-0234.1,https://journals.ametsoc.org/view/journals/mwr...,144.0,4279-4287,...,NaN,DOI,Mt5lL4qoeDgJ,https://journals.ametsoc.org/view/journals/mwr...,10.1175/mwr-d-16-0234.1,4,LN Boisvert,(2016),The impact of the extreme winter 2015/16 Arcti...,[AIRX3STD]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5362,0,PWEPTUQK,0.0,webpage,NaN,NaN,NaN,https://bia.unibz.it/esploro/outputs/doctoral/...,NaN,NaN,...,NaN,Null,Lo69bVOtgEUJ,https://bia.unibz.it/esploro/outputs/doctoral/...,,5369,CF Brunello,(2019),Seasonal hydroclimate across the Himalayas: in...,[TRMM_2B31]
5363,0,J3LZDR95,0.0,journalArticle,"[{'firstName': 'Janek', 'lastName': 'Walk', 'c...","[{'tag': 'Alluvial fan', 'type': 1}, {'tag': '...",10.1016/j.gloplacha.2019.102994,https://www.sciencedirect.com/science/article/...,185,102994,...,NaN,DOI,5hhinLLt9bcJ,https://www.sciencedirect.com/science/article/...,,5370,J Walk,(2020),"Gradients in climate, geology, and topography ...",[TRMM_2B31]
5364,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Null,TMHbdA4dZZsJ,https://search.proquest.com/openview/d6dcd3f25...,,5371,A Racoviteanu,(2011),"Himalayan glaciers: combining remote sensing, ...",[TRMM_2B31]
5365,0,ZB8ZMSAQ,0.0,journalArticle,"[{'firstName': 'Pedro', 'lastName': 'Val', 'cr...","[{'tag': 'erosion rate', 'type': 1}, {'tag': '...",10.1016/j.epsl.2017.11.004,https://www.sciencedirect.com/science/article/...,482,160-170,...,NaN,DOI,BczI6ZJBzRwJ,https://www.sciencedirect.com/science/article/...,,5372,P Val,(2018),Tectonic control of erosion in the southern Ce...,[TRMM_2B31]


In [27]:
export_df.columns

Index(['Unnamed: 0', 'key', 'version', 'itemType', 'creators', 'tags', 'DOI',
       'url', 'volume', 'pages', 'title', 'publicationTitle', 'abstractNote',
       'date', 'ISSN', 'language', 'journalAbbreviation', 'libraryCatalog',
       'accessDate', 'bib', 'index', 'tURL', 'fail', 'perf', 'year', 'author',
       'rights', 'issue', 'shortTitle', 'numPages', 'extra', 'ISBN',
       'publisher', 'proceedingsTitle', 'conferenceName', 'series', 'place',
       'university', 'thesisType', 'bookTitle', 'edition', 'institution',
       'reportType', 'reportNumber', 'websiteType', 'websiteTitle',
       'seriesNumber', 'archiveLocation', 'callNumber', 'DOI_null',
       'result_id_o', 'link_o', 'pub_doi_o', 'index_o', 'author_o', 'year_o',
       'title_o', 'ShortNames_o'],
      dtype='object')

In [28]:
export_df['title'] = np.where(export_df['title'].isna(), export_df['title_o'], export_df['title'])
export_df['year'] = np.where(export_df['year'].isna(), export_df['year_o'], export_df['year'])
export_df['author'] = np.where(export_df['author'].isna(), export_df['author_o'], export_df['author'])
export_df['url'] = np.where(export_df['url'].isna(), export_df['link_o'], export_df['url'])
export_df['DOI'] = np.where(export_df['DOI'].isna(), export_df['pub_doi_o'], export_df['DOI'])

In [29]:
export_df

,Unnamed: 0,key,version,itemType,creators,tags,DOI,url,volume,pages,...,callNumber,DOI_null,result_id_o,link_o,pub_doi_o,index_o,author_o,year_o,title_o,ShortNames_o
0,0,XZFWVNZT,0.0,journalArticle,"[{'firstName': 'G.', 'lastName': 'Abdelaziz', ...",[],10.1088/1742-6596/869/1/012070,https://doi.org/10.1088/1742-6596/869/1/012070,869.0,12070,...,NaN,DOI,bZ5KnGFtrEAJ,https://iopscience.iop.org/article/10.1088/174...,10.1088/1742-6596/869/1/012070,0,G Abdelaziz,(2017),Search for Best Astronomical Observatory Sites...,"[AIRX3STD, MAT1NXSLV, MAT1NXFLX]"
1,0,6EUEUZ3I,0.0,journalArticle,"[{'firstName': 'Zhong', 'lastName': 'Liu', 'cr...",[{'tag': 'tropical meteorology and climatology...,10.3390/rs12172821,https://www.mdpi.com/2072-4292/12/17/2821,12.0,2821,...,NaN,DOI,9n-nmxo8geAJ,https://www.mdpi.com/813170,,1,Z Liu,(2020),NASA global satellite and model data products ...,"[AIRX3STD, OMAEROe, NLDAS_NOAH0125_H, OMDOAO3e..."
2,0,ZRGZ7FQD,0.0,journalArticle,"[{'firstName': 'Abhay', 'lastName': 'Devasthal...",[],,https://www.jstor.org/stable/26243653,97.0,ES255-ES263,...,NaN,Null,ixJUtElCtKAJ,https://www.jstor.org/stable/26243653,,2,A Devasthale,(2016),SUPPLEMENT: A DECADE OF SPACEBORNE OBSERVATION...,[AIRX3STD]
3,0,3HST9T9G,0.0,journalArticle,"[{'firstName': 'Guanheng', 'lastName': 'Zheng'...","[{'tag': 'Satellite remote sensing', 'type': 1...",10.1016/j.rse.2019.111269,https://www.sciencedirect.com/science/article/...,231.0,111269,...,NaN,DOI,tFBynLLhJPsJ,https://www.sciencedirect.com/science/article/...,,3,G Zheng,(2019),Satellite-based simulation of soil freezing/th...,[AIRX3STD]
4,0,E8NR9S8V,0.0,journalArticle,"[{'firstName': 'Linette N.', 'lastName': 'Bois...",[],10.1175/MWR-D-16-0234.1,https://journals.ametsoc.org/view/journals/mwr...,144.0,4279-4287,...,NaN,DOI,Mt5lL4qoeDgJ,https://journals.ametsoc.org/view/journals/mwr...,10.1175/mwr-d-16-0234.1,4,LN Boisvert,(2016),The impact of the extreme winter 2015/16 Arcti...,[AIRX3STD]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5362,0,PWEPTUQK,0.0,webpage,NaN,NaN,,https://bia.unibz.it/esploro/outputs/doctoral/...,NaN,NaN,...,NaN,Null,Lo69bVOtgEUJ,https://bia.unibz.it/esploro/outputs/doctoral/...,,5369,CF Brunello,(2019),Seasonal hydroclimate across the Himalayas: in...,[TRMM_2B31]
5363,0,J3LZDR95,0.0,journalArticle,"[{'firstName': 'Janek', 'lastName': 'Walk', 'c...","[{'tag': 'Alluvial fan', 'type': 1}, {'tag': '...",10.1016/j.gloplacha.2019.102994,https://www.sciencedirect.com/science/article/...,185,102994,...,NaN,DOI,5hhinLLt9bcJ,https://www.sciencedirect.com/science/article/...,,5370,J Walk,(2020),"Gradients in climate, geology, and topography ...",[TRMM_2B31]
5364,0,NaN,NaN,NaN,NaN,NaN,,https://search.proquest.com/openview/d6dcd3f25...,NaN,NaN,...,NaN,Null,TMHbdA4dZZsJ,https://search.proquest.com/openview/d6dcd3f25...,,5371,A Racoviteanu,(2011),"Himalayan glaciers: combining remote sensing, ...",[TRMM_2B31]
5365,0,ZB8ZMSAQ,0.0,journalArticle,"[{'firstName': 'Pedro', 'lastName': 'Val', 'cr...","[{'tag': 'erosion rate', 'type': 1}, {'tag': '...",10.1016/j.epsl.2017.11.004,https://www.sciencedirect.com/science/article/...,482,160-170,...,NaN,DOI,BczI6ZJBzRwJ,https://www.sciencedirect.com/science/article/...,,5372,P Val,(2018),Tectonic control of erosion in the southern Ce...,[TRMM_2B31]


In [30]:
export_df.to_json('zot_translator_output.json', orient='records')